In [13]:
import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

In [14]:
df = pd.read_csv('data.csv')
# df.head()

In [16]:
# we need to predict MSRP - retail price - based on other features
# but before we need to look at data, prepare and beautify it...

# STEP 1 - make columns look good:
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(" ", "_")
# df.head()

In [17]:
# STEP 2 - use dtypes and make all df values look unified

df.dtypes # for example number_of_doors is float64 )) -- not a good idea )) 

make                  object
model                 object
year                   int64
engine_fuel_type      object
engine_hp            float64
engine_cylinders     float64
transmission_type     object
driven_wheels         object
number_of_doors      float64
market_category       object
vehicle_size          object
vehicle_style         object
highway_mpg            int64
city_mpg               int64
popularity             int64
msrp                   int64
dtype: object